In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import os
import os.path
import statistics as stat

In [ ]:
def SMEP(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1, chance level
    sig1 = 0.5  # Prior belief variance at trial 1
    sigO = 0.25  # Observation variance
    sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    choices= df['choice'].tolist()
    rewards = df['outcome'].tolist()

    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter
    persev = theta[2]  # # Perseveration bonus parameter

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []


    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error

            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb  # Calculate utilities for all options
            
            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def main_SMEP():

    nll = []
    betaList = []
    phiList = []
    persevList = []
    IDlist = []


    global number
    for number in range (1,410):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 10
        bnds = ((0.1,10),(-5,5), (0,1),)

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0.01,0.2)
            persev = np.random.uniform(0.01,0.2) ## starting from low perseveration

            startParams = [beta, phi, persev]
            theta_optim = minimize(SMEP, startParams, method = "L-BFGS-B", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
        persevList.append(theta[2])
     

        dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'perseveration': persevList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('bayesian learner neg phi.csv')


In [ ]:
main_SMEP()

In [ ]:
def bayesian_RL(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1
    sig1 = 0.5  # Prior belief variance at trial 1
    sigO = 0.25  # Observation variance
    # sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    choices= df['choice'].tolist()
    rewards = df['outcome'].tolist()

    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter
    persev = theta[2]  # # Perseveration bonus parameter
    sigD = theta[3]  ### diffusion variance as a free parameter

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []


    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error

            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb  # Calculate utilities for all options
            
            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def main_bayesian_RL():

    nll = []
    betaList = []
    phiList = []
    persevList = []
    volatList = []
    IDlist = []


    global number
    for number in range (401,410):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 30
        bnds = ((0.1,10),(0.01,10), (0,1),(0,1))

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0.01,0.2)
            persev = np.random.uniform(0.01,0.2) ## starting from low perseveration
            sigD = np.random.uniform(0.01,0.5)

            startParams = [beta, phi, persev,sigD]
            theta_optim = minimize(bayesian_RL, startParams, method = "L-BFGS-B", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
        persevList.append(theta[2])
        volatList.append(theta[3])
     

        dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'perseveration': persevList,'diffusion variance': volatList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('Bayesian RL model parameters_new400.csv')


In [ ]:
main_bayesian_RL()

In [ ]:
def bayesian_RL_sig0(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1
    sig1 = 0.5  # Prior belief variance at trial 1
    # sigO = 0.25  # Observation variance
    # sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    choices= df['choice'].tolist()
    rewards = df['outcome'].tolist()

    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter
    persev = theta[2]  # # Perseveration bonus parameter
    sigD = theta[3]  ### diffusion variance as a free parameter
    sigO = theta[4]  ###observation variance as a free paramter

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []


    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error

            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb  # Calculate utilities for all options
            
            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def main_bayesian_RL2():

    nll = []
    betaList = []
    phiList = []
    persevList = []
    volatList = []
    IDlist = []
    stochList = []


    global number
    for number in range (1,401):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 30
        bnds = ((0.1,10),(0.01,10), (0,1),(0,1),(0,1))

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0.01,0.2)
            persev = np.random.uniform(0.01,0.2) ## starting from low perseveration
            sigD = np.random.uniform(0.01,0.5)
            sigO = np.random.uniform(0.01,0.5)

            startParams = [beta, phi, persev,sigD, sigO]
            theta_optim = minimize(bayesian_RL, startParams, method = "L-BFGS-B", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
        persevList.append(theta[2])
        volatList.append(theta[3])
        stochList.append(theta[4])
     

        dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'perseveration': persevList,'diffusion variance': volatList,'observation variance': stochList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('Bayesian RL model parameters_new.csv')


In [ ]:
main_bayesian_RL2()

In [ ]:
def SMEP_nopersev(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1
    sig1 = 0.25  # Prior belief variance at trial 1
    sigO = 0.25  # Observation variance
    sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    choices= df['choice'].tolist()
    rewards = df['outcome'].tolist()

    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []

    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            # pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error
            # print (pe)
            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            # print (Kgain)
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            # print (v)
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb  # Calculate utilities for all options
            # print (utilities)
            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            # print (probabilities)
            # print (choices[t])
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def SMERP(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1
    sig1 = 0.5  # Prior belief variance at trial 1
    sigO = 0.25  # Observation variance
    sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    choices= df['choice'].tolist()
    rewards = df['outcome'].tolist()

    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter
    persev = theta[2]  # # Perseveration bonus parameter
    gamma = theta[3]

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []

    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error
            # print (pe)
            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            # print (Kgain)
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            # print (v)
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Calculate the total uncertainty
            total_uncertainty = np.sum(sig)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb  # Calculate utilities for all options

            # Include the new term for total uncertainty-based random exploration
            utilities += (gamma * v) / total_uncertainty

            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            # print (probabilities)
            # print (choices[t])
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def SMEP_with_L2(params):
    beta, phi, persev = params
    original_SMEP_value = SMEP(params)  # Your original SMEP function calculation
    
    # L2 Regularization Term
    lambda_reg = 0.001  # Regularization strength, adjust based on your needs
    l2_penalty = lambda_reg * (beta**2 + phi**2 + persev**2)
    
    # Modified Objective Function with L2 Regularization
    return original_SMEP_value + l2_penalty

In [ ]:
def SMERP_with_L2(params):
    beta, phi, persev, gamma = params
    original_SMEP_value = SMERP(params)  # Your original SMEP function calculation
    
    # L2 Regularization Term
    lambda_reg = 0.5  # Regularization strength, adjust based on your needs
    l2_penalty = lambda_reg * (beta**2 + phi**2 + persev**2 + gamma**2)
    
    # Modified Objective Function with L2 Regularization
    return original_SMEP_value + l2_penalty

In [ ]:
def SMEP_with_attractor(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1
    sig1 = 0.5  # Prior belief variance at trial 1
    sigO = 0.25  # Observation variance
    sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    choices= df['choice'].tolist()
    rewards = df['outcome'].tolist()

    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter
    persev = theta[2]  # # Perseveration bonus parameter
    attractor = theta[3] ## attractor strength

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []

    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error
            # print (pe)
            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            # print (Kgain)
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            # print (v)
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Calculate attractor bonus for each option based on its uncertainty
            attractor_bonus = attractor / (sig + 1e-6)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb + attractor_bonus # Calculate utilities for all options
            # print (utilities)
            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            # print (probabilities)
            # print (choices[t])
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def main_SMEP_with_attractors():

    nll = []
    betaList = []
    phiList = []
    persevList = []
    attractList = []
    IDlist = []


    global number
    for number in range (1,337):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 30
        bnds = ((0.1,10),(0.1,5), (0,1), (0,1),)

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0,1)
            persev = np.random.uniform(0,0.2) ## starting from low perseveration
            attractor = np.random.uniform(0.4,0.6)

            startParams = [beta, phi, persev, attractor]
            theta_optim = minimize(SMEP_with_attractor, startParams, method = "L-BFGS-B", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
        persevList.append(theta[2])
        attractList.append(theta[3])
     

        # dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'perseveration': persevList, 'model likelihood': nll}
        # data_df = pd.DataFrame(dataDict)
        # data_df.to_csv('SMEP model parameters_negPhi_negRho2.csv')


In [ ]:
main_SMEP_with_attractors()

In [ ]:
def main_SMEP_nopersev():

    nll = []
    betaList = []
    phiList = []
    IDlist = []


    global number
    for number in range (1,337):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 30
        bnds = ((0.1,10),(0.01,10))

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0.5,2)

            startParams = [beta, phi]
            theta_optim = minimize(SMEP_nopersev, startParams, method = "TNC", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
     

        # dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'model likelihood': nll}
        # data_df = pd.DataFrame(dataDict)
        # data_df.to_csv('SMEP model parameters no perseveration.csv')


In [ ]:
def main_SMERP():

    nll = []
    betaList = []
    phiList = []
    persevList = []
    gammaList = []
    IDlist = []


    global number
    for number in range (1,337):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 30
        bnds = ((0.1,10),(0.01,10), (0,1),(0,1))

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0.5,2)
            persev = np.random.uniform(0,0.2) ## starting from low perseveration
            gamma = np.random.uniform(0.1,0.3)

            startParams = [beta, phi, persev,gamma]
            theta_optim = minimize(SMERP_with_L2, startParams, method = "L-BFGS-B", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
        persevList.append(theta[2])
        gammaList.append(theta[3])
     

        # dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'perseveration': persevList, 'model likelihood': nll}
        # data_df = pd.DataFrame(dataDict)
        # data_df.to_csv('SMEP model parameters with L2.csv')


### run each model here

In [ ]:
main_SMERP()

In [ ]:
main_SMEP()

In [ ]:
main_SMEP_nopersev()

### this version of the classify_trials function classifies trials into exploitation, random exploration, and directed exploration trials

In [ ]:
def classify_trials(choices, rewards, phi, persev):
    n_trials = len(rewards)
    n_options = len(np.unique(choices))

    # Initialize beliefs about mean rewards and uncertainties for each option
    v = np.full(n_options, 0.5)  # Assuming binary rewards, 0.5 is a neutral initial belief
    sig = np.full(n_options, np.sqrt(0.5 * (1 - 0.5)))  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25

    # label list
    # exploit is 1, random  is 2, directed is 3
    labelList = [2]  # first trial is random exploration

    for t in range(1, n_trials):  # Start from 2nd trial as 1st choice is assumed random
        eb = phi * sig  # Exploration bonus
        utilities = v + eb

        # Add perseveration bonus if applicable
        if t > 1:
            utilities[choices[t-1]] += persev

        # Classify trials
        chosen_option = choices[t]
        best_option = np.argmax(v)

        if chosen_option == best_option:  ## exploitation trials
            labelList.append(1)
        else:
            most_uncertain_option = np.argmax(eb)
            if chosen_option == most_uncertain_option:  ## directed exploration
                labelList.append (3)
            else:                           ## random exploration
                labelList.append (2)

        # Update beliefs for chosen option using a simplified model
        pe = rewards[t] - v[chosen_option]
        Kgain = sig[chosen_option]**2 / (sig[chosen_option]**2 + sig0**2)  # Assuming sigO=0.25 for binary rewards
        v[chosen_option] += Kgain * pe  # Update the mean belief
        sig[chosen_option] = np.sqrt((1 - Kgain) * sig[chosen_option]**2)  # Update the uncertainty

    return labelList

In [ ]:
def trial_by_trial_variables(choices, rewards, phi, persev, ID, session):
    n_trials = len(rewards)
    n_options = len(np.unique(choices))

    # Initialize beliefs about mean rewards and uncertainties for each option
    v = np.full(n_options, 0.5)  # Assuming binary rewards, 0.5 is a neutral initial belief
    sig = np.full(n_options, np.sqrt(0.5 * (1 - 0.5)))  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25

    
    # store RPE and uncertainty bonus of chosen choice
    PElist = []
    kgainList = [] 
    uncertainty_relative = []
    uncertainty_total = []
    value_relative = []
    utilitiesList = []
    ebList = []

    

    for t in range(n_trials):  # Start from 2nd trial as 1st choice is assumed random
        
        chosen_option = choices[t]

        # Update beliefs for chosen option using a simplified model
        pe = rewards[t] - v[chosen_option]
        PElist.append(pe)

        Kgain = sig[chosen_option]**2 / (sig[chosen_option]**2 + sig0**2)  # Assuming sigO=0.25 for binary rewards
        kgainList.append(Kgain)
        
        v[chosen_option] += Kgain * pe  # Update the mean belief
        unchosenValue = np.delete(v, chosen_option)
        value_relative.append(v[chosen_option]-np.mean(unchosenValue))
        
        sig[chosen_option] = np.sqrt((1 - Kgain) * sig[chosen_option]**2)  # Update the uncertainty
        unchosenUncertainty =  np.delete(sig, chosen_option)
        uncertainty_relative.append(sig[chosen_option] - np.mean(unchosenUncertainty))
        uncertainty_total.append(np.sum(sig))

        eb = phi * sig  # Exploration bonus
        utilities = v + eb
        ebList.append(eb[chosen_option])

        # Add perseveration bonus if applicable
        if t > 1:
            utilities[chosen_option] += persev

        utilitiesList.append(utilities[chosen_option])

    trialNum = np.linspace(1, len(PElist),len(PElist))


    trialDF = pd.DataFrame({'ID': [ID]*len(PElist), 'session': [session]*len(PElist), 'trial number': trialNum,'prediction error': PElist, 'Kalman gain': kgainList, 'relative uncertainty': uncertainty_relative, 'total uncertainty': uncertainty_total, 'relative value diff': value_relative, 'utilities': utilitiesList, 'exploration bonus':ebList})
        
    ## drop the first row
    trialDF = trialDF.iloc[1:, :]
            

    return trialDF

In [ ]:
def trial_by_trial_variables2(choices, rewards, phi, persev, ID, session):
    n_trials = len(rewards)
    n_options = len(np.unique(choices))

    # Initialize beliefs about mean rewards and uncertainties for each option
    v = np.full(n_options, 0.5)  # Assuming binary rewards, 0.5 is a neutral initial belief
    sig = np.full(n_options, np.sqrt(0.5 * (1 - 0.5)))  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25
    sig = np.full(3, 0.5)  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25
    decayV = 0.98  # Decay parameter for value
    decayE = 0.98
    decay_center = 0.5  # Decay center
    sigD = 0.1  # Diffusion variance

    
    # store RPE and uncertainty bonus of chosen choice
    PElist = []
    kgainList = [] 
    uncertaintyList = []
    uncertainty_total = []
    valueList = []
    utilitiesList = []

    

    for t in range(n_trials):  # Start from 2nd trial as 1st choice is assumed random
        
        chosen_option = choices[t]

        # Update beliefs for chosen option using a simplified model
        pe = rewards[t] - v[chosen_option]
        PElist.append(pe)

        Kgain = sig[chosen_option]**2 / (sig[chosen_option]**2 + sig0**2)  # Assuming sigO=0.25 for binary rewards
        kgainList.append(Kgain)
        
        v[chosen_option] += Kgain * pe  # Update the mean belief
        sig[chosen_option] = np.sqrt((1 - Kgain) * sig[chosen_option]**2)  # Update the uncertainty

        eb = phi * sig  # Exploration bonus
   
        utilities = v + eb

        # Add perseveration bonus if applicable
        if t > 1:
            utilities[chosen_option] += persev

        utilitiesList.append(utilities)

        # Apply decay to beliefs and uncertainties
        v = decayV * v + (1 - decayV) * decay_center
        sig = np.sqrt(decayE**2 * sig**2 + sigD**2)

        valueList.append(v[chosen_option])
        uncertaintyList.append(sig[chosen_option])
        uncertainty_total.append(np.sum(sig))

    trialNum = np.linspace(1, len(PElist),len(PElist))


    trialDF = pd.DataFrame({'ID': [ID]*len(PElist), 'session': [session]*len(PElist), 'trial number': trialNum,'prediction error': PElist, 'Kalman gain': kgainList, 'uncertainty': uncertaintyList,'total uncertain': uncertainty_total, 'value': valueList})
        
    ## drop the first row
    trialDF = trialDF.iloc[1:, :]
            

    return trialDF

In [ ]:
def trial_by_trial_variables3(choices, rewards, phi, persev, ID, session):
    n_trials = len(rewards)
    n_options = len(np.unique(choices))

    # Initialize beliefs about mean rewards and uncertainties for each option
    v = np.full(n_options, 0.5)  # Assuming binary rewards, 0.5 is a neutral initial belief
    sig = np.full(n_options, np.sqrt(0.5 * (1 - 0.5)))  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25
    sig = np.full(3, 0.5)  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25
    decayV = 0.98  # Decay parameter for value
    decayE = 0.98
    decay_center = 0.5  # Decay center
    sigD = 0.1  # Diffusion variance

    
    # store RPE and uncertainty bonus of chosen choice
    PElist = []
    kgainList = [] 
    uncertaintyList1 = []
    uncertaintyList2= []
    uncertaintyList3 = []
    uncertainty_total = []
    valueList = []
    utilitiesList = []

    

    for t in range(n_trials):  # Start from 2nd trial as 1st choice is assumed random
        
        chosen_option = choices[t]

        # Update beliefs for chosen option using a simplified model
        pe = rewards[t] - v[chosen_option]
        PElist.append(pe)

        Kgain = sig[chosen_option]**2 / (sig[chosen_option]**2 + sig0**2)  # Assuming sigO=0.25 for binary rewards
        kgainList.append(Kgain)
        
        v[chosen_option] += Kgain * pe  # Update the mean belief
        sig[chosen_option] = np.sqrt((1 - Kgain) * sig[chosen_option]**2)  # Update the uncertainty

        eb = phi * sig  # Exploration bonus
   
        utilities = v + eb

        # Add perseveration bonus if applicable
        if t > 1:
            utilities[chosen_option] += persev

        utilitiesList.append(utilities)

        # Apply decay to beliefs and uncertainties
        v = decayV * v + (1 - decayV) * decay_center
        sig = np.sqrt(decayE**2 * sig**2 + sigD**2)

        valueList.append(v[chosen_option])
        uncertaintyList1.append(sig[0])
        uncertaintyList2.append(sig[1])
        uncertaintyList3.append(sig[2])
        uncertainty_total.append(np.sum(sig))

    trialNum = np.linspace(1, len(PElist),len(PElist))


    trialDF = pd.DataFrame({'ID': [ID]*len(PElist), 'session': [session]*len(PElist), 'trial number': trialNum,'prediction error': PElist, 'Kalman gain': kgainList, 'uncertainty1': uncertaintyList1,'uncertainty2': uncertaintyList2,'uncertainty3': uncertaintyList3,'total uncertain': uncertainty_total, 'value': valueList})
        
    ## drop the first row
    trialDF = trialDF.iloc[1:, :]
            

    return trialDF

In [ ]:
datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
# SMEPPath = '/Users/sijinchen/Desktop/Cathy Caroline paper/SMEP label/'   ### new folder to build here
trialPath = '/Users/sijinchen/Desktop/Cathy Caroline paper/test trial varying uncertainy/'

paramDF = pd.read_csv('/Users/sijinchen/Desktop/Cathy Caroline paper/results files/SMEP model parameters.csv')
phi = paramDF['SMEP_directed exploration_phi'].tolist()
persev = paramDF['SMEP_perseveratiobonus'].tolist()
ID = paramDF['ID'].tolist()
session = paramDF['session_type'].tolist()

directed_overall  = []
random_overall = []
exploit= []
random_ore = []
directed_ore = []
exploit_abs= []
random_abs = []
directed_abs = []

for num_id in range (1, 410):
    df = pd.read_csv(datapath +  str(num_id) +'.csv')
    print (num_id)

    choice = df['choice'].tolist()
    print (len(choice))
    outcome = df['outcome'].tolist()
    
    trialDF = trial_by_trial_variables3(choice, outcome,phi[num_id-1], persev[num_id-1],ID[num_id-1],session[num_id-1])
    print (len(trialDF))
    trialDF.to_csv(trialPath +'//' + str(ID[num_id-1]) +'_' + str(session[num_id-1]) +  '_trial by trial measures4.csv')
    
    

In [ ]:
datapath = '/Users/sijinchen/Desktop/new data/new summary data_RL/'
SMEPPath = '/Users/sijinchen/Desktop/new data/SMEP label new/'   ### new folder to build here

paramDF = pd.read_csv('/Users/sijinchen/Desktop/new data/SMEP model parameters_new.csv')
phi = paramDF['SMEP_directed exploration_phi'].tolist()
persev = paramDF['SMEP_perseveratiobonus'].tolist()
ID = paramDF['ID'].tolist()
session = paramDF['session_type'].tolist()


directed_overall  = []
random_overall = []
exploit= []
random_ore = []
directed_ore = []
exploit_abs= []
random_abs = []
directed_abs = []

for num_id in range (1, 74):
    df = pd.read_csv(datapath + '//' + str(num_id) +'.csv')

    choice = df['choice'].tolist()
    outcome = df['outcome'].tolist()
    

    labelList = classify_trials(choice, outcome, phi[num_id-1], persev[num_id-1])
    SMEPdict = {'SMEP label': labelList}
    SMEPdf = pd.DataFrame(SMEPdict)
    SMEPdf.to_csv(SMEPPath +'//' + ID[num_id-1] +'_' + session[num_id-1] +  '_SMEP label.csv')

    num_exploit = labelList.count(1)
    exploit_abs.append(num_exploit)
    non_exploit = len(choice)-num_exploit
    exploit.append(num_exploit/len(choice))

    directed_overall.append(labelList.count(3)/len(choice))
    random_overall.append(labelList.count(2)/len(choice))

    directed_ore.append(labelList.count(3)/non_exploit)
    random_ore.append(labelList.count(2)/non_exploit)

    directed_abs.append(labelList.count(3))
    random_abs.append(labelList.count(2))
    


resultDF = pd.DataFrame({'ID': ID, 'session': session, 'exploit trial percent': exploit, 'random exploration percent overall': random_overall, 'directed exploration percent overall': directed_overall, 'random exploration over all explore': random_ore, 'directed exploration over all explore': directed_ore, 'exploit trial num': exploit_abs, 'random exploration trial num': random_abs, 'directed exploration trial num': directed_abs})
resultDF.to_csv('percent of SMEP trial type_new.csv')
    
    

## this version of classify_trials function takes into account perserveration trials

In [ ]:
def classify_trials_with_persev(choices, rewards, phi, persev):
    n_trials = len(rewards)
    n_options = len(np.unique(choices))

    # Initialize beliefs about mean rewards and uncertainties for each option
    v = np.full(n_options, 0.5)  # Assuming binary rewards, 0.5 is a neutral initial belief
    sig = np.full(n_options, np.sqrt(0.5 * (1 - 0.5)))  # Initial uncertainty based on Bernoulli variance
    sig0 = 0.25

    # label list
    # exploit is 1, random  is 2, directed is 3, perserveration is 4
    labelList = [2]  # first trial is random exploration

    for t in range(1, n_trials):  # Start from 2nd trial as 1st choice is assumed random

        chosen_option = choices[t]
        previous_option = choices[t-1]

        # Update beliefs based on previous choice and outcome
        pe = rewards[t-1] - v[previous_option]
        Kgain = sig[previous_option]**2 / (sig[previous_option]**2 + sig0**2)  # Assuming sigO=0.25 for binary rewards
        v[previous_option] += Kgain * pe  # Update the mean belief
        sig[previous_option] = np.sqrt((1 - Kgain) * sig[previous_option]**2)  # Update the uncertainty

        eb = phi * sig  # Exploration bonus
        utilities = v + eb

        # Add perseveration bonus if applicable
        if t > 1:
            utilities[choices[t-1]] += persev

        # Classify current trials
        best_option = np.argmax(v)
        most_uncertain_option = np.argmax(eb)
        
        if chosen_option == best_option:  ## exploitation trials
            labelList.append(1)
        elif chosen_option != best_option and chosen_option == previous_option:
            labelList.append(4)
        else:
            # prev_best = 0   ## reset this to 0
            if chosen_option == most_uncertain_option:  ## directed exploration
                labelList.append (3)
            else:                           ## random exploration
                labelList.append (2)

        

    return labelList

In [ ]:
datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
SMEPPath = '/Users/sijinchen/Desktop/Cathy Caroline paper/SMEP label/'   ### new folder to build here

paramDF = pd.read_csv('/Users/sijinchen/Desktop/Cathy Caroline paper/python code/SMEP model parameters.csv')
phi = paramDF['directed exploration'].tolist()
persev = paramDF['perseveration'].tolist()
ID = paramDF['ID'].tolist()
session = paramDF['session_type'].tolist()

directed_overall  = []
random_overall = []
exploit_overall= []
persev_overall = []

random_ore = []
directed_ore = []

exploit_abs= []
random_abs = []
directed_abs = []
persev_abs = []

for num_id in range (1, 337):
    df = pd.read_csv(datapath + '//' + str(num_id) +'.csv')

    choice = df['choice'].tolist()
    outcome = df['outcome'].tolist()

    labelList = classify_trials_with_persev(choice, outcome, phi[num_id-1], persev[num_id-1])
    SMEPdict = {'SMEP label': labelList}
    SMEPdf = pd.DataFrame(SMEPdict)
    SMEPdf.to_csv(SMEPPath +'//' + ID[num_id-1] +'_' + session[num_id-1] +  '_SMEP label.csv')

    num_exploit = labelList.count(1)
    num_persev = labelList.count(4)
    exploit_abs.append(num_exploit)
    non_exploit = len(choice)-num_exploit-num_persev

    exploit_overall.append(num_exploit/len(choice))
    directed_overall.append(labelList.count(3)/len(choice))
    random_overall.append(labelList.count(2)/len(choice))
    persev_overall.append(labelList.count(4)/len(choice))

    directed_ore.append(labelList.count(3)/non_exploit)
    random_ore.append(labelList.count(2)/non_exploit)

    directed_abs.append(labelList.count(3))
    random_abs.append(labelList.count(2))
    persev_abs.append(labelList.count(4))
    


resultDF = pd.DataFrame({'ID': ID, 'session': session, 'exploit trial percent': exploit_overall, 'random exploration percent overall': random_overall, 'directed exploration percent overall': directed_overall, 'perserveration percent overall': persev_overall, 'random exploration over all explore': random_ore, 'directed exploration over all explore': directed_ore, 'exploit trial num': exploit_abs, 'random exploration trial num': random_abs, 'directed exploration trial num': directed_abs, 'perserveration trial num': persev_abs})
resultDF.to_csv('percent of SMEP trial type_with perserv.csv')
    
    

## modify the code so that we are only looking at first 200 trials for BL2

In [ ]:
def SMEP(theta):
    v1 = 0.5  # Prior belief mean reward value at trial 1
    sig1 = 0.5  # Prior belief variance at trial 1
    sigO = 0.25  # Observation variance
    sigD = 0.1  # Diffusion variance
    decay = 0.99  # Decay parameter
    decay_center = 0.5  # Decay center

    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')
    user_id = df['user id'].tolist()[0]
    session = df['session_type'].tolist()[0]

    if session == 'BL2' or session == '6M' or session == '7M' or session == '12M':
        choices= df['choice'].tolist()[:200]
        rewards = df['outcome'].tolist()[:200]
    else:
        choices= df['choice'].tolist()
        rewards = df['outcome'].tolist()
        
    beta = theta[0]  # Exploration bonus parameter
    phi = theta[1]  # Uncertainty influence parameter
    persev = theta[2]  # # Perseveration bonus parameter

    # Initial values for reward belief and uncertainty
    v = np.full(3, v1)  # Initial beliefs about mean rewards for each option
    sig = np.full(3, sig1)  # Initial uncertainties for each option

    # Store probability given model, use to calculate model likelihood
    probList = []

    for t in range(len(rewards)):

        # Exploration and Perseveration Bonuses
        eb = phi * sig  # Exploration bonus based on current uncertainty
        pb = np.zeros(3)  # Perseveration bonus initialized to 0 for all options

        if t == 0:  ### first trial
             pChoice = 1/3   ### first choice is random, p (choice) is 0.33
             probList.append(pChoice)

        else:
            pb[choices[t-1]] = persev  # Apply perseveration bonus to the previous choice, only tracking one trial back

            # Update beliefs using a simplified Kalman filter approach
            pe = rewards[t-1] - v[choices[t-1]]  # Prediction error
            # print (pe)
            Kgain = sig[choices[t-1]]**2 / (sig[choices[t-1]]**2 + sigO**2)  # Kalman gain
            # print (Kgain)
            v[choices[t-1]] += Kgain * pe  # Update the mean belief
            # print (v)
            sig[choices[t-1]] = np.sqrt((1 - Kgain) * sig[choices[t-1]]**2)  # Update the uncertainty

            # Apply decay to beliefs and uncertainties
            v = decay * v + (1 - decay) * decay_center
            sig = np.sqrt(decay**2 * sig**2 + sigD**2)

            # Action Selection based on the current Utility (value + bonuses)
            utilities = v + eb + pb  # Calculate utilities for all options
            # print (utilities)
            probabilities = np.exp(utilities*beta) / np.sum(np.exp(utilities*beta))  # Softmax transformation to probabilities
            # print (probabilities)
            # print (choices[t])
            pChoice = probabilities[choices[t]]
            probList.append(pChoice)

    ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike


In [ ]:
def main_SMEP():

    nll = []
    betaList = []
    phiList = []
    persevList = []
    IDlist = []


    global number
    for number in range (1,337):
        IDlist.append (number)
        print ("*************************************************")
        print ("This is ID number " + str(number))


        replication = 30
        bnds = ((0.1,10),(0.01,10), (0,1),)

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            beta = np.random.uniform(0.5,2)  ## starting from moderate weight
            phi = np.random.uniform(0.5,2)
            persev = np.random.uniform(0,0.2) ## starting from low perseveration

            startParams = [beta, phi, persev]
            theta_optim = minimize(SMEP_with_L2, startParams, method = "L-BFGS-B", bounds = bnds, tol = 0.0005)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        betaList.append(theta[0])
        phiList.append(theta[1])
        persevList.append(theta[2])
     

        dataDict = { 'numeric ID': IDlist,'random exploration': betaList, 'directed exploration': phiList, 'perseveration': persevList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('SMEP model parameters_200 trials only.csv')


In [ ]:
main_SMEP()